In [22]:
import tensorflow as tf
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import time

class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.data = data
        self.targets = targets
        self.batch_size = batch_size
        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch  


class binaryClassification:
    #def __init__(self):

    def __init__(self, _regLambda = 0.001, _batchSize = 500, \
                 _learningRate = 0.001, _learningRateArr = None, _numIter = 5000, _gd = 0, _err =0, _qn=0):
        self.regLambda = _regLambda #_regLambda
        #self.regLambda = tf.cast(self.regLambda, dtype = tf.float64)

        self.batchSize = _batchSize
        self.learningRateArr = _learningRateArr #we need to tune
        self.learningRate = _learningRate
        self.numIter = _numIter
        #self.numEpoch =int(np.ceil(trainData.shape[0]/7)) 

        
        self.gd =  _gd  #0 for normal GD, 1 for adam 
        self.err = _err #for type of error to minimize, 1 for default corssEntropy, 0 MSE error
        self.qn = _qn #question type, logistic = 1, linear = 0
        
        
        #optional    
        self.crossEntropyArr = []
        self.mseLossArr = []

    
    def linearMSE(self):
        '''
        y_hat, target will be fed

        shape = [dim by 1] for binary classification
        '''
        #print("y_hat",  y_hat)
        y_hat = tf.placeholder(tf.float64, shape=[None,1 ], name='y_hat')
        target = tf.placeholder(tf.float64, shape=[None, 1], name='target')
    
        se_mat = tf.square(tf.subtract(y_hat, target))
        #print("msemst", mse_mat)
        mse_mat = tf.reduce_mean(se_mat)
        loss = tf.reduce_mean(mse_mat)
        loss = tf.div(loss, tf.constant(2.0, dtype = tf.float64))

        return y_hat, target, loss 
    
    def linearMSENoFeed(self, y_hat, target):
        '''
        a tensor function, no feed
        '''
        print("y_hat",  y_hat.shape)
        print("target", target.shape)
    
        y_hat = tf.convert_to_tensor(y_hat)
        target = tf.convert_to_tensor(target)
        
        target = tf.cast(target, dtype = 'float64')
        se_mat = tf.square(tf.subtract(y_hat, target))
        #print("msemst", mse_mat)
        mse_mat = tf.reduce_mean(se_mat)

        loss = tf.reduce_mean(mse_mat)

        loss = loss/2.0 #tf.div(loss, tf.constant(2.0))
        #print(mse_mat.eval())

        return loss  
    
    def crossEntropyError(self, y_hat, y_target):
        crossEntropyErrorCurr = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_target, logits= y_hat))
        return crossEntropyErrorCurr
    
    def buildGraphBinary(self):
        '''
        Input: _data is x in the equation, dim by 784 flattened tensor
               _target is y in the equaion
               _regLambda is the wegithed decay coeff
               _learningRate is the epsilon
               err is the type of error, 1 default corssEntropy
        '''

        #declare using a placeholder, feed in _data and _target to x ,y 
        X = tf.placeholder(tf.float64, shape=[None, 784], name='dataX')
        # W initialize to a gaussian distr, honestly anything would work
        W = tf.Variable(tf.truncated_normal(shape=[784, 1], stddev=0.1), name='weights')

        W = tf.cast(W, dtype=tf.float64)
        b = tf.Variable(0.0, name='biases')
        b = tf.cast(b, dtype=tf.float64)

        y_target = tf.placeholder(tf.float64, shape=[None, 1], name='targetY')
        

        #compute the current y_hat
        if self.qn == 1: #logistic:
            wtxb =  tf.matmul(X, W) + b
            y_hat = tf.sigmoid(wtxb)
            regTerm = tf.multiply( tf.constant(0.50, dtype = tf.float64),\
                                  tf.multiply(tf.constant(self.regLambda,dtype = tf.float64), tf.reduce_mean(tf.square(W))))
            crossEntropyErrorCurr = self.crossEntropyError(y_hat, y_target)
            crossEntropyErrorCurr = tf.add(crossEntropyErrorCurr, regTerm)  
            errTerm = crossEntropyErrorCurr
            
        else:
            y_hat =  tf.matmul(X, W) + b
            mseCurr = self.linearMSENoFeed(y_hat, y_target)
            regTerm = tf.multiply( tf.constant(0.50, dtype = tf.float64), tf.multiply(self.regLambda, tf.reduce_mean(tf.square(W))))
            mseCurr = tf.add(mseCurr, regTerm)
            errTerm = mseCurr
        

        if self.gd == 0:
            optimizer = tf.train.GradientDescentOptimizer(learning_rate = self.learningRate)
        else:
            optimizer = tf.train.AdamOptimizer(learning_rate = self.learningRate)
    
        if self.err == 1:
            train = optimizer.minimize(loss=crossEntropyErrorCurr)
        else:
            train = optimizer.minimize(loss=mseCurr)

        return W, b, errTerm, y_hat, X, y_target, train
    
    def plotFig(self, _dim, y , addInfo, title="default", xLabel="xlabel", yLabel="yLabel", plotLabel ="plotLabel", _num =1 ):
        x = np.linspace(0, _dim, num=_dim)
        y = np.array(y)
        print("$$$$$$$$$$$$$$$ in plot fig$$$$$$$$$$$$$$$$$")
        print(y.shape)
        plt.figure(_num)
        plt.title(title)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        for i in range(y.shape[0]):
            #print("x", x)
            #print("y", y[i])
            plt.plot(x, y[i], label = plotLabel + str(addInfo[i]))
        
        plt.legend()
        plt.savefig( title + str(_num) + ".png")
        plt.close()
        plt.clf()

              
    def runLogisticGraphPart1(self, trainData, trainTarget, validData, validTarget,testData, testTarget, plotTitleArr):
        '''
        Input: _data,
        Output:
           required accuracy/epoch plots
           
        '''
        trainLossArr= []
        mseLossArr = []
        
            
        for learningRate in self.learningRateArr:
            trainLossL = []
            mseLossL = []
            tf.reset_default_graph()
            W, b, errTerm, y_hat, X, y_target, train = self.buildGraphBinary()
            y_hat_mse, target_mse, mseLoss = self.linearMSE()
    
            init = tf.global_variables_initializer()
            sess = tf.InteractiveSession()
            sess.run(init)
            initialW = sess.run(W)  
            initialb = sess.run(b)            
            #training model and iter through batches
            print("learningrate = ", learningRate)
            trainBatchSampler = BatchSampler(trainData, trainTarget, self.batchSize)

            for i in range(self.numIter):
                dataBatch, targetBatch = trainBatchSampler.get_batch()
                #dataBatch = tf.stack(dataBatch)
                #targetBatch = tf.stack(targetBatch)
                currentW, currentb, errTrain, y_predict, trainModel = sess.run([W, b, errTerm, y_hat, train], feed_dict={X: dataBatch, y_target: targetBatch})
                trainLossL.append(errTrain)
            
                mseLossVal = sess.run(mseLoss, feed_dict={y_hat_mse: y_predict, target_mse: targetBatch})
                mseLossL.append(mseLossVal)
            
                if i%3500 == 0:
                    print("current entropy", errTrain)
                    #print("current mse", mseLoss)
                    print("epoch ", i/500)
        
            trainLossArr.append(trainLossL)
            mseLossArr.append(mseLossL)
        
        print("train done")
        #print("mselossall", mseLossAll)
        
        for i in range(len(plotTitleArr)):
            self.plotFig(self.numIter, trainLossArr, self.learningRateArr,  title = plotTitleArr[i],\
            plotLabel="learning rate", _num = i)

class loadData:
    def __init__(self):
        self.flatten = True
        self.addOnes = False
    
    def arrFlatten(self, arr):
        '''
        type np array
        '''
        dataDim1, dum1, dum2 = arr.shape
        arr = np.reshape(arr, [ dataDim1 ,784 ])
        return arr   
    
    def loadBinData(self):
    # import binary NOTMIST data set
        with np.load("notMNIST.npz") as data :
            Data, Target = data ["images"], data["labels"]
            posClass = 2
            negClass = 9
            dataIndx = (Target==posClass) + (Target==negClass)
            Data = Data[dataIndx]/255.
            Target = Target[dataIndx].reshape(-1, 1)
            Target[Target==posClass] = 1
            Target[Target==negClass] = 0
            
            if self.flatten:
                Data = self.arrFlatten(Data)
        
            if self.addOnes:
                Data = np.concatenate((np.ones((Data.shape[0], 1)),Data),axis=1)
        
        
            np.random.seed(521)
            randIndx = np.arange(len(Data))
            np.random.shuffle(randIndx)
            Data, Target = Data[randIndx], Target[randIndx]
            trainData, trainTarget = Data[:3500], Target[:3500]
            validData, validTarget = Data[3500:3600], Target[3500:3600]
            testData, testTarget = Data[3600:], Target[3600:]
        
        print("Data binary class Loaded")
        print("-------------------------------")
        return trainData, trainTarget,validData, validTarget, testData, testTarget
    
    
if __name__ == '__main__':
    dataLoader = loadData()
    trainData, trainTarget, validData, validTarget,testData, testTarget = dataLoader.loadBinData()
    
    
    plotTitleArr = ["q2-3 Adam Opt lambda = 0 linear loss vs number of epoches"]
    linearClassifier = binaryClassification(_regLambda = 0.001, _batchSize = 500, \
                 _learningRate = 0.001, _learningRateArr = [0.001], _numIter = 5000, _gd =1, _err=0, _qn=0)
    
    #plotTitleArr = ["Q2-1 logistic loss vs number of epoches", "Q2-1 MSE loss vs number of epoches"]
    #logisticClassifier = binaryClassification(_regLambda = 0.001, _batchSize = 500, \
    #             _learningRate = 0.001, _learningRateArr = [0.001], _numIter = 5000, _gd =1, _err=1, _qn=1)
    #logisticClassifier.runLogisticGraphPart1(trainData, trainTarget, validData, validTarget,testData, testTarget)


Data binary class Loaded
-------------------------------
